In [1]:
"""
    Author: Iris Shakya
    course: Big Data Technology
    Project: 2, tf-idf w/ similarity search( mapreduce + pyspark)
    instructor: Prof. Lei Xei
    References: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.zipWithIndex.html
                https://towardsdatascience.com/tf-idf-calculation-using-map-reduce-algorithm-in-pyspark-e89b5758e64c
"""


'\n    Author: Iris Shakya\n    course: Big Data Technology\n    Project: 2, tf-idf w/ similarity search( mapreduce + pyspark)\n    instructor: Prof. Lei Xei\n    References: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.zipWithIndex.html\n                https://towardsdatascience.com/tf-idf-calculation-using-map-reduce-algorithm-in-pyspark-e89b5758e64c\n'

In [2]:
import math
from pyspark import SparkConf, SparkContext
import numpy as np
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession
import gc
from pyspark.sql.functions import monotonically_increasing_id

sc = SparkContext(master = 'local')
spark = SparkSession.builder.appName('pySpark word-count').config('spark.some.config.option', 'some-value')             .getOrCreate()

22/11/18 11:56:01 WARN Utils: Your hostname, Iriss-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 146.95.218.233 instead (on interface en0)
22/11/18 11:56:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/18 11:56:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.sparkContext.textFile("/Users/iris/project2_test2.txt")

import regex as re
regex_st = r'\d{8}'

# doc = data.map(lambda x: x.split(regex_st))
doc = data.flatMap(lambda x: x.split())
doc1 = doc.map(lambda x: (x.split(regex_st)))

In [4]:
my_para = data.map(lambda x: tuple(x.split('\n')))

In [5]:
#split_list = my_para.map(lambda x: list(x[0].split()))
split_list = my_para.map(lambda x: tuple(x[0].split()))
#split_list2 = split_list.filter(lambda x: x[0] is not None)


doc_id = split_list.zipWithIndex().keyBy(lambda x: x[1])

# unpacking 2 tuple into 1 singular tuple to compute
doc_id = doc_id.map(lambda x: tuple((x[0], *x[1])))

# re-packing the tuple
doc_id = doc_id.map(lambda x: tuple((x[0], x[1])))
#split_list.take(3)


[('9415305',
  "'adenobody'",
  'approach',
  'viral',
  'targeting',
  'specific',
  'enhanced',
  'adenoviral',
  'gene',
  'delivery',
  'recombinant',
  'adenovirus',
  'enormous',
  'potential',
  'vector',
  'gene',
  'therapy',
  'evolve',
  'efficient',
  'method',
  'dis_infection_dis',
  'wide',
  'host',
  'range',
  'lead',
  'concern',
  'specificity',
  'gene',
  'delivery',
  'order',
  'target',
  'adenovirus',
  'type',
  '5-based',
  'vector',
  'investigate',
  'antibody',
  'approach',
  'virus',
  'neutralize',
  'scfv',
  'antibody',
  'fragment',
  'isolate',
  'phage',
  'library',
  'c-terminal',
  'fusion',
  'protein',
  'gene_epidermal_growth_factor_gene',
  'gene_egf_gene',
  'construct',
  'fusion',
  'protein',
  "'adenobody'",
  'bind',
  'both',
  'fibre',
  'protein',
  'adenovirus',
  'gene_egf_receptor_gene',
  'gene_egfr+_gene',
  'human',
  'cell',
  'direct',
  'adenoviral',
  'binding',
  'new',
  'receptor',
  'using',
  'system',
  'efficiency'

In [6]:
# separate into key-value pairs
# ('doc_id', 'term') --> (( 'doc_id', 'token'), 1 )
kv_mapper = doc_id.flatMap(lambda x: tuple( [ ((x[0], i),1) for i in x[1]]))

In [7]:
#kv_mapper.take(5)

[((0, '9415305'), 1),
 ((0, "'adenobody'"), 1),
 ((0, 'approach'), 1),
 ((0, 'viral'), 1),
 ((0, 'targeting'), 1)]

In [8]:
# do the term-count
# (('doc_id', 'token'), 1) --> (('doc_id', 'token'), [1++])
reduce_1 = kv_mapper.reduceByKey(lambda x, y: x+y)


#N = kv_mapper.map(lambda x: x[1]).sum()
#reduce_1.take(4)

[((0, '9415305'), 1),
 ((0, "'adenobody'"), 3),
 ((0, 'approach'), 3),
 ((0, 'viral'), 1)]

In [9]:

# tf = reducer_2.map(lambda x: (x[0], (len(doc_id1)/x[1])))
# tf.take(5)

In [10]:
# mapping for IDF start
# (('doc_id', 'token'), tf) --> ('token', ('doc_id', tf))


# tf = reducer_2.map(lambda x: (x[0], (len(doc_id1)/x[1])))

shuffle = reduce_1.map(lambda x: (x[0][1], (x[0][0], x[1])))
#shuffle.take(3)

[('9415305', (0, 1)), ("'adenobody'", (0, 3)), ('approach', (0, 3))]

In [11]:
# tokenise for idf
# (('doc_id', 'token'), tf) --> ('token', ('doc_id', tf, 1))
mapper_2 = reduce_1.map(lambda x: (x[0][1], (x[0][0], x[1],1)))
#mapper_2.take(2)

[('9415305', (0, 1, 1)), ("'adenobody'", (0, 3, 1))]

In [12]:
# only take term and token
# ('token', ('doc_id', tf,1)) --> ('token', 1)
mapper_3 = mapper_2.map(lambda x: (x[0], x[1][2]))
#mapper_3.take(3)

[('9415305', 1), ("'adenobody'", 1), ('approach', 1)]

In [13]:
# counting by token
# ('token', 1) --> ('token', [1++])
reducer_2 = mapper_3.reduceByKey(lambda x, y: x+y)
#reducer_2.take(3)

[('9415305', 1), ("'adenobody'", 1), ('approach', 857)]

In [14]:
# convert into python list so we can pass it into a map transformation
doc_id1 = doc_id.map(list).collect()
print(len(doc_id1))

10694


In [15]:
doc_id2 = split_list.map(lambda x: x[0]).count()
doc_id2

10694

In [16]:
# tf = reducer_2.map(lambda x: (x[0], x[1], (x[1]/len(doc_id2[1]))))


tf = reducer_2.map(lambda x: (x[0], x[1], (x[1]/doc_id2)))
#tf.take(5)

[('9415305', 1, 9.35103796521414e-05),
 ("'adenobody'", 1, 9.35103796521414e-05),
 ('approach', 857, 0.08013839536188518),
 ('viral', 91, 0.008509444548344866),
 ('targeting', 414, 0.03871329717598653)]

In [17]:
# idf = tf.map(lambda x: (x[0], math.log10(len(doc_id1)/x[1])))
tf_copy = tf.map(lambda x: (x[0], x[2]))
idf = tf.map(lambda x: (x[0], math.log10(doc_id2/x[1])))


#idf.take(3)

[('9415305', 4.029140179764322),
 ("'adenobody'", 4.029140179764322),
 ('approach', 1.0961593578411237)]

In [18]:
#tf_copy.take(3)

[('9415305', 9.35103796521414e-05),
 ("'adenobody'", 9.35103796521414e-05),
 ('approach', 0.08013839536188518)]

In [19]:
joint_my = tf_copy.join(idf)
#joint_my.take(4)

[('9415305', (9.35103796521414e-05, 4.029140179764322)),
 ('approach', (0.08013839536188518, 1.0961593578411237)),
 ('specific', (0.1525154292126426, 0.816686218724046)),
 ('enhanced', (0.006919768094258463, 2.159908460033346))]

In [20]:
# ('doc_id', 'everything_else') --> (('doc_id'), ('token', 'tf', 'idf', 'tf-idf'))
# tf_idf = joint_my.map(lambda x: (x[1][0][0], ( x[0], x[1][0][1], x[1][1], x[1][0][1] * x[1][1])) ).sortByKey()
tf_idf = joint_my.map(lambda x: (x[0], ( x[1][0] * x[1][1])) )
#tf_idf.collect()

[('9415305', 0.00037676642788145895),
 ('approach', 0.08784445199830214),
 ('specific', 0.124557249180748),
 ('enhanced', 0.014946065648257676),
 ('gene', 0.1524624417131081),
 ('delivery', 0.027175158576358275),
 ('potential', 0.13608920361348023),
 ('vector', 0.027823508359970668),
 ('therapy', 0.15143651809443662),
 ('host', 0.01684488392115063),
 ('concern', 0.01956789895347076),
 ('specificity', 0.04088246293546533),
 ('type', 0.11276558099356443),
 ('investigate', 0.14053282977277248),
 ('antibody', 0.13113669073629874),
 ('isolate', 0.043014667880766494),
 ('phage', 0.00794094659926597),
 ('library', 0.013967732014208884),
 ('c-terminal', 0.01330367057882437),
 ('fusion', 0.027044728694965783),
 ('gene_epidermal_growth_factor_gene', 0.15069440470560003),
 ('gene_egf_gene', 0.10384299548079017),
 ('construct', 0.03609194961760183),
 ('bind', 0.06707810038755825),
 ('both', 0.15548458853590127),
 ('gene_egf_receptor_gene', 0.09662366599730492),
 ('human', 0.1583572114816084),
 ('b

In [21]:
prefix1 = 'dis_breast_cancer_dis'
prefix2 = 'gene_egfr+_gene'
# doc_1 = tf_idf.filter(lambda x: x[0] == prefix1).map(lambda y: (y[0], y[1][3]))
# doc_2 = tf_idf.filter(lambda x: x[1][0] == prefix2).map(lambda y: (y[0], y[1][3]))

doc_1 = tf_idf.filter(lambda x: x[0] == prefix1)
doc_2 = tf_idf.filter(lambda x: x[0] == prefix2)

In [22]:
#doc_2.take(5)

[('gene_egfr+_gene', 0.1589594509228879)]